In [6]:
import pandas as pd
import numpy as np
import re

import re
import jellyfish
from collections import Counter

from tqdm.notebook import tqdm
tqdm.pandas(desc="Example Desc")

## Подготавливаем данные по категориям:

In [366]:
shops = ['okey.csv', 'ashan.csv',  'perecrestok.csv', 'azbuka_vkusa.csv', 'metro.csv', 'utkonos.csv']
result = pd.DataFrame(columns=['Категория', 'Название'])

for shop in shops:
    df = pd.read_csv(f'data/{shop}', skiprows=1, encoding='cp1251', delimiter=';')
    df = df[['Категория', 'Название']]
    result = pd.concat([result, df]).dropna().drop_duplicates()

In [149]:
result.sample(10)

,Категория,Название
8191,Каталог/Товары для мам и детей/Товары для школ...,набор карандашей bic kids tropicolors 2 12 цветов
32019,"Хлеб, торты, сладости/Конфеты, шоколад/Конфеты...","жевательные конфеты skittles 2 в 1, 38г"
36229,Домашняя страница/Скидки в Санкт-Петербурге/Бо...,подгузники pampers active baby-dry 5-9 кг 3 ра...
20368,Главная/Все для праздника/Идеи подарков/Подарк...,"сковорода-гриль «нева металл посуда титан», съ..."
15249,"Творог, Сырки, Десерты/Творог",творог из козьего молока ефремовское молоко «к...
67299,Главная/Ремонт/Сантехника/Комплектующие,угольник комбинированный внутренняя резьба d20...
17056,"Чай, кофе, какао/Чай/Mlesna","чай mlesna ""dimbula"" (димбула) черный, высокор..."
14876,"Крупы, масло, консервы, соусы, орехи/Батончики...","батончик мюсли ""диyes"" лесная ягода без сахара..."
62143,Аптека/Лекарственные препараты/Дыхательная сис...,эриспирус таб.пролонг.п.п.о. 80мг n20
43449,"Канцтовары, все для праздника, журналы/Журналы",журнал садовод и огородник 21/18


In [372]:
# Нужно удалить ненужные позиции из категорий, такие как: Главная, Домашняя страница, Каталог:
# А также для увеличения точности, подготовим данные под конкретную задачу и удалим записи с категориями про Игрушки, Аптека, Лекарственные препараты и т.п.
counter.most_common(10)

[('Главная', 214392),
 ('Домашняя страница', 94312),
 ('Каталог', 76688),
 ('Аптека', 54228),
 ('Товары для дома', 53588),
 ('Игрушки', 36580),
 ('Косметика, личная гигиена', 27440),
 ('Детское питание, уход, игрушки', 26668),
 ('Лекарственные препараты', 25444),
 ('Красота и здоровье', 24088)]

In [163]:
def remove_stop_word_from_catagory(x):
    x = x.replace('Главная/', '').replace('Домашняя страница/', '').replace('Каталог/', '')
    return x

result['Категория'] = result['Категория'].apply(remove_stop_word_from_catagory)

In [ ]:
# Удаляем нерелевантные категории:

result['Категория'] = result['Категория'][result['Категория'].str.find('Аптека') == -1]
result['Категория'] = result['Категория'][result['Категория'].str.find('Игрушки') == -1]
result['Категория'] = result['Категория'][result['Категория'].str.find('Лекарственные препараты') == -1]
result['Категория'] = result['Категория'][result['Категория'].str.find('Канцтовары, все для праздника, журналы') == -1]
result['Категория'] = result['Категория'][result['Категория'].str.find('Астигматические или торические контактные линзы') == -1]
result['Категория'] = result['Категория'][result['Категория'].str.find('Сервировка стола') == -1]
result['Категория'].dropna(inplace=True)

In [ ]:
# Сохраняем
result.csv('data/category.csv', index=False)
# result.to_excel('data/category.xlsx', index=False)

## Находим нужнуую категорию:

In [82]:
df = pd.read_csv('pyat_and_perek.csv')
df = df.dropna()
df.head()

,name,clear_name,size,brand,quantity,price,sum
0,1:3182857 Яйца куриные столовые 10шт,яйца куриные столовые,10шт,None,3.0,55.9,167.7
1,2*3669625 Кондиционер LENOR 910мл,кондиционер,910мл,lenor,1.0,119.9,119.9
2,3:3500750 Средство FAIRY 450мл,средство,450мл,fairy,1.0,94.9,94.9
3,4*3281576 Порошок DOSIA COLOR 400г,порошок color,400г,dosia,1.0,39.9,39.9
4,5*70709 БЗМЖ Молоко PARMALAT 1л,бзмж молоко,1л,parmalat,2.0,59.9,119.8


In [ ]:
# Удаляем бзмж:
df['clear_name'] = df['clear_name'].apply(lambda x: x.replace('бзмж', ''))

In [349]:
def generate_categoty(name: str) -> str:
    """Ищет подходящую категорию для товара.
    Это простая версия, использующая жадный алгоритм, можно оптимизировать.
    
    Логика такая:
    Сначала в собранном датасете ищем полное вхождение названия товара, это хорошо работает
    с названиями, состоящими из 1-2 слова, например, `яйцо`. Затем группируем по категорям
    (т.к. категорий может быть уйму) и находим наиболее часто встречающуюся категорию.
    Её и отдаём.
    Если не получилось найти подстроку, то для названия товара находим сходство Джаро
    (расстояние Левенштейна работает плохо) для каждой позиции из собранного датасета,
    ищем максимальное сходство и возращаем найденую категорию по товару.
    
    
    Я использоал разные подходы, это пока что лучшее, что получилось. Можно ещё получше почистить
    датасет категорий и убрать те товары, которые не продаются в пятёрочке или перекрёстке.
    Для оптимизации стоит использовать кэширование, а то у нас сложность почти O(N^3).
    Или другой алгоритм.
    """
    
    tmp = pd.DataFrame(result[result['Название'].str.find(name) != -1])
    if len(tmp):
        return tmp.groupby('Категория').count().sort_values(by='Название', ascending=False).index[0]
    
    res = result.iloc[np.argmax(result['Название'].str.lower() \
                          .apply(lambda x: jellyfish.jaro_distance(x, name)))]['Категория']
    
    return res
    

In [357]:
# ВНИМАНИЕ! Это будет долго работать!
%%time
df['category'] = df['clear_name'].progress_apply(generate_categoty)


CPU times: user 1h 48min 31s, sys: 13min 35s, total: 2h 2min 7s
Wall time: 2h 3min 26s


In [363]:
# Смотрим, что получилось:
df.sample(10)

,name,clear_name,size,brand,quantity,price,sum,category,first_word
14927,*3423871 BIM.Ср.COLOR д/ст.жид.1500г,color д/ст жид,1500г,bimbo,1.0,199.99,199.99,"Стирка, уборка, уход за обувью/Стирка и уход з...",color
4266,6:3625801 Капуста брокколи 900г,капуста брокколи,900г,None,1.0,139.00,139.00,Замороженные продукты/Овощи и смеси,капуста
4779,12*: 3421454 NIV.Дезод.ЭФФ.ПУД.спр.жен.1,дезод эфф пуд спр жен,1,nivea,1.0,109.90,109.90,Косметика и гигиена/Уход за телом/Дезодоранты ...,дезод
11794,*3683407 БЛ.ГОР.Сосис.МОЛ.ГОСТ вар.500г,гор сосиски мол гост вареная(ый),500г,благо,1.0,179.99,179.99,"Хлеб, торты, сладости/Хлеб, хлебцы, выпечка/Хл...",гор
12428,*3469348 РУБЛ.Ветч.Д/ЗАВТРАКА в/у 400г,ветч д/завтрака в/у,400г,рублевская усадьба,1.0,139.99,139.99,Домашняя страница,ветч
15455,*32120 LAC.З/п ACTIV 50мл,з/п activ,50мл,lacalut,1.0,129.99,129.99,Канцтовары/Товары для школы/Школьные пеналы,з/п
14781,3275468 F.H.Икр.мойв.ар/копч.с/с 180г,икр мойв ар/копч с/с,180г,frau hellen,1.0,124.99,124.99,"Хлеб и выпечка/Булочки, пироги, сладкая выпечк...",икр
3862,15: 3682402 Томаты ЧЕРРИ МЕДОВЫЕ 200г,томаты черри медовые,200г,None,1.0,165.00,165.00,"Овощи, фрукты, ягоды, зелень, грибы/Овощи/РОСТ",томаты
13313,3908230 СЕВ.ПФ Желудок ЦБ мыш.охл.550г,желудок мыш охлаждённая,550г,север,1.0,116.99,116.99,"Рыба, икра, морепродукты/Охлажденная рыба",желудок
10014,*3693306 БЗМЖ Морож.ЧИСТАЯ ЛИНИЯ 200г,мороженное линия,200г,чистая линия,2.0,89.90,179.80,"Мясо, птица, колбасы/Мясо и птица",мороженное


In [361]:
# Берём 1 слово:
df['first_word'] = df['clear_name'].apply(lambda x: x.split()[0])

In [364]:
# Сохраняем:
df.to_excel('with_category.xlsx', index=False)